<a href="https://colab.research.google.com/github/Anikrage/EEG-Classification-using-Self-Supervised-Diffusion-Convolutional-Graph-Neural-Network/blob/prototype/Res2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import mne

EEG CSV INTO MNE RAW

In [ ]:
def load_eeg_csv_to_mne(csv_file, sfreq, electrode_names, group_column, condition_column):
    """
    Load EEG data from a CSV file into an MNE RawArray.

    Args:
        csv_file (str): Path to the CSV file.
        sfreq (float): Sampling frequency of the EEG data.
        electrode_names (list): List of electrode names (column headers for EEG data).
        group_column (str): Name of the column with group labels (H or MDD).
        condition_column (str): Name of the column with conditions (EC, EO, TASK).

    Returns:
        raw (mne.io.RawArray): MNE RawArray object containing EEG data.
    """
    # Load the CSV
    df = pd.read_csv(csv_file)

    # Extract EEG data and transpose to match MNE's format (channels x time)
    eeg_data = df[electrode_names].values.T

    # Define MNE info structure
    info = mne.create_info(ch_names=electrode_names, sfreq=sfreq, ch_types='eeg')

    # Create RawArray
    raw = mne.io.RawArray(eeg_data, info)

    # Add annotations for group and condition (optional)
    annotations = []
    for _, row in df.iterrows():
        annotations.append(
            f"Group:{row[group_column]}, Condition:{row[condition_column]}"
        )
    raw.set_annotations(mne.Annotations(onset=np.arange(len(annotations)) / sfreq,
                                         duration=1/sfreq,
                                         description=annotations))

    return raw

# Define parameters
csv_file = '/content/drive/MyDrive/eeg_mdd_dataset/eeg_dataset.csv'
sfreq = 256  # Example sampling frequency (Hz)
electrode_names = [
    "Fp1-LE", "F3-LE", "C3-LE", "P3-LE", "O1-LE", "F7-LE", "T3-LE", "T5-LE",
    "Fz-LE", "Fp2-LE", "F4-LE", "C4-LE", "P4-LE", "O2-LE", "F8-LE", "T4-LE",
    "T6-LE", "Cz-LE", "Pz-LE", "A2-A1", "23A-23R", "24A-24R"
]
group_column = "Group"
condition_column = "Condition"

# Load the CSV into an MNE RawArray
raw = load_eeg_csv_to_mne(csv_file, sfreq, electrode_names, group_column, condition_column)

# Plot the raw data (optional)
raw.plot(scalings='auto', show=True)


Creating RawArray with float64 data, n_channels=22, n_times=1048575
    Range : 0 ... 1048574 =      0.000 ...  4095.992 secs
Ready.
Using matplotlib as 2D backend.


KeyboardInterrupt: 

Preprocessing with MNE

In [ ]:
# Apply a band-pass filter (e.g., 0.5–45 Hz)
raw_filtered = raw.copy().filter(l_freq=0.5, h_freq=45)

In [ ]:
# Run ICA to remove artifacts (e.g., eye blinks, muscle noise)
ica = mne.preprocessing.ICA(n_components=15, random_state=42)
ica.fit(raw_filtered)

# Plot ICA components for inspection
ica.plot_components()

In [ ]:
# Automatically find and remove artifacts
ica.detect_artifacts(raw_filtered)
raw_cleaned = ica.apply(raw_filtered)